## Setup

In [ ]:
import pickle
from tqdm.auto import tqdm as progressbar
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as c
import importlib
importlib.reload(c)
import common as c
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients
%matplotlib qt5

In [ ]:
with open("output.pkl", "rb") as f:
    res = pickle.load(f)

board, aruco_dict = load_board("board.pkl")

images=[]
for data in progressbar(res):
    hdr = c.hdr(list(c.decode(d) for d in data["pictures"]), data["exposures"])
    images.append((data["point"], hdr))

In [ ]:
c.animate((g[1] for g in images))

## Funcs

In [ ]:
def mask_rgb(img, mask):
	ret = np.zeros_like(img)
	for i in range(ret.shape[-1]):
		ret[:,:,i] = img[:,:,i] * mask
	return ret

def get_charuco_rect(board):
	delim = np.max(board.chessboardCorners, axis=0) + board.getSquareLength() * np.array([1, 1, 0])
	corner_points = [
		[0, 0, 0],
		[delim[0], 0, 0],
		delim,
		[0, delim[1], 0],
	]
	return np.array(corner_points)

def get_charuco_mask(image, board, rvec, tvec, mtx, dist):
	if image.ndim==2:
		mask = np.zeros_like(image)
	elif image.ndim==3:
		mask = np.zeros(image.shape[:2])

	rect = get_charuco_rect(board)
	points, _ = cv2.projectPoints(rect, rvec, tvec, mtx, dist)
	proj_points_round = np.round(points, 0).astype(np.int32)
	cv2.fillPoly(mask, [proj_points_round], True, 255 )
	return mask

def crop_charuco_board(image, board, rvec, tvec, mtx, dist):
	mask = get_charuco_mask(image, board, rvec, tvec, mtx, dist)
	if image.ndim == 3:
		return mask_rgb(image, mask)
	elif image.ndim==2:
		return mask*image

def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

## Calib

In [ ]:
idxs, mtx, dist, rvecs, tvecs = calibrate_charuco_local((d[1] for d in images), board, aruco_dict,)

# discard boards with not enough points
images = [ images[i] for i in idxs ]

### Calib with prior

In [ ]:
plot = False
idxs, mtx, dist, rvecs, tvecs = calibrate_charuco_local((d[1] for d in images), board, aruco_dict, prior=(mtx,dist), plot=plot)
# Once again remove images with too little markers
images = [images[i] for i in idxs ]

for i, image in enumerate(images):
    images[i] = image[0], cv2.undistort(image[1], mtx, dist, None, mtx)

# Laser

## Inspecionar Laser

In [ ]:
def find_board_and_laser(image, rvec, tvec):
    """Crop out the image to contain only charuco board, and find the laser points in this image
    @param image original image
    @param rvec rotation vec of the board
    @param tvec translation vec of the board
    @returns: (masked, centroids), tuple containing the masked image with only the board, and vector of laser points """
    masked = crop_charuco_board(image, board, rvec, tvec, mtx, dist)
    centroids = c.column_centroids(c.red_contrast(masked), mask=c.red_contrast(masked)>30)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T
    return masked, p_centroids

### Plot de todas as linhas

In [ ]:
n=0
for (point, image), rvec, tvec in zip(images, rvecs, tvecs):
    masked, p_centroids = find_board_and_laser(image, rvec, tvec)
    plt.plot(p_centroids[1], label=n)
    n+=1
plt.legend() 

### Inspecionar linha específica

In [ ]:
i = 19
(point, image), rvec, tvec = images[i], rvecs[i], tvecs[i]
masked, p_centroids = find_board_and_laser(image, rvec, tvec)
plt.imshow(masked)
plt.plot(p_centroids[0],p_centroids[1])

### Animar Laser

In [ ]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(images[0][1], animated=True)
    pointsplot, = plt.plot(range(images[0][1].shape[0]), "r", animated=True)
    return pointsplot, imgplot

def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    (point, image), rvec, tvec = images[i], rvecs[i], tvecs[i]
    masked, centroids = find_board_and_laser(image, rvec, tvec)

    # Update anim
    imgplot.set_data(masked)
    pointsplot.set_data(centroids[0], centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=200, interval=20, blit=True)

## Calibrar

In [ ]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

### Achar todos os pontos 3D

In [ ]:
from collections import defaultdict
Xlist = []

zcounter = defaultdict(int)
for (point, image), rvec, tvec in zip(images, rvecs, tvecs):
	# count z 
	zcounter[point[-1]] += 1

	masked, p_centroids = find_board_and_laser(image, rvec, tvec)
	n = chess_plane(rvec, tvec)
	print(point[-1])
	print(tvec)
	print(n)
	print("=====")
	points3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
	Xlist.append(points3d)
print(zcounter)

In [ ]:
X = np.hstack(Xlist)

### Mostrar pontos 3D

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")

### Fit num plano

In [ ]:
u, s, vt = np.linalg.svd(X.T, full_matrices=False)
v = vt.T
n = v[:,-1]
print(n)

### Plot Plano e pontos

In [ ]:
x0, xf = np.min(X[0]), np.max(X[0])
y0, yf = np.min(X[1]), np.max(X[1])
xplot = np.linspace(x0,xf)
yplot = np.linspace(y0,yf)
xplot, yplot = np.meshgrid(xplot, yplot)
a,b,c,d = n
zplot = -(a*xplot + b*yplot + d)/c

# Resultado

## Plot

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[0],X[1], X[2], color="r")
ax.plot_surface(xplot, yplot, zplot, color="b", alpha=0.5)

## Params

In [ ]:
print("=== Intrinsic Camera Params ===")
print("===== Camera Matrix =====")
print(mtx)
print("===== Lens Distortion Params =====")
print(dist.T)
print("=== Laser Params ===")
print("==== Laser Plane 4-vec ====")
print(n)
print("==== Laser Plane Equation ====")
print(f"{1e3*a:.2f}x + {1e3*b:.2f}y + {1e3*c:.2f}z + {1e3*d:.2f} = 0")

## Save

In [ ]:
calib = dict(
    mtx = mtx,
    dist= dist,
    n=n
)
with open("calib.pkl", "wb") as f:
    pickle.dump(calib, f)